In [2]:
# import random
import numpy as np
# import matplotlib.pyplot as plt
# import plotly.express as px
# import time
import os
# import cv2
from PIL import Image, ImageDraw, ImageOps, ImageColor


In [3]:
def gen_noise(x, y, rgb=True):
    if rgb:
        fig_r = np.random.randint(256, size=(x, y), dtype=np.uint8)
        fig_g = np.random.randint(256, size=(x, y), dtype=np.uint8)
        fig_b = np.random.randint(256, size=(x, y), dtype=np.uint8)
        fig = np.stack((fig_r, fig_g, fig_b), axis=2)
    else:
        fig = np.random.randint(256, size=(x, y), dtype=np.uint8)
    return fig


In [4]:
# def gen_noise_rgb(x, y):
#     # X = np.shape(img)[0]
#     # Y = np.shape(img)[1]
#     # for y in range(Y):
#     #     for x in range(X):
#     #         bright = random.randint(0, 255)
#     #         img[x][y] = [bright, bright, bright]
#     fig_r = np.random.randint(256, size=(x, y), dtype=np.uint8)
#     fig_g = np.random.randint(256, size=(x, y), dtype=np.uint8)
#     fig_b = np.random.randint(256, size=(x, y), dtype=np.uint8)
#     fig = np.stack((fig_r, fig_g, fig_b), axis=2)
#     return fig

In [5]:
def gen_frame(mx, rgb: bool=True):
    fig = Image.fromarray(mx)
    if rgb:
        fig = fig.convert('RGBA')
    else:
        fig.convert('L')
    return fig


In [6]:
def gen_gif(x: int, y: int, n_frames: int=100, rgb: bool=False):
    if not os.path.exists('./images'):
        os.mkdir('images')
    if not os.path.exists('./videos'):
        os.mkdir('videos')

    gif = []
    for _ in range(n_frames):
        if rgb:
            img = gen_noise(x, y, rgb)
        else:
            img = gen_noise(x, y, rgb)
        fig = gen_frame(img)
        gif.append(fig)

    return gif

In [7]:
def save_gif(gif, n_frames, framerate_hz: int=30, rgb: bool=False):
    if rgb:
        fname = f'static_rgb{framerate_hz:02d}'
    else:
        fname = f'static{framerate_hz:02d}'
    gif[0].save(f'./videos/{fname}.gif',
        save_all=True, optimize=True, append_images=gif[1:n_frames],
        loop=0, duration=1000/framerate_hz)

In [8]:
# Create black and white gif
# gif = gen_gif(400, 500)
# save_gif(gif, 100)


In [9]:
# Create RGB gif
# gif = gen_gif(400, 500, rgb=True)
# save_gif(gif, 100)

In [10]:

# alpha = Image.new('1', (500, 500))
# draw = ImageDraw.Draw(alpha)
# draw.ellipse((50, 50, 450, 450), fill='blue', outline='blue')
# alpha_inv = ImageOps.invert(alpha.convert('RGB'))
# alpha_inv = alpha_inv.convert('1')

# gif = []
# for _ in range(100):
#     fig1 = gen_frame(gen_noise(500, 500))
#     fig1.putalpha(alpha)
#     fig1 = fig1.convert('RGBA')
#     # fig1.show()
#     fig2 = gen_frame(gen_noise(500, 500, rgb=False), rgb=False)
#     fig2.putalpha(alpha_inv)
#     fig2 = fig2.convert('RGBA')
#     # fig2.show()
#     fig2.paste(fig1, (0,0), alpha)
#     fig = fig2
#     gif.append(fig)
#     # fig.show()

# save_gif(gif, 100, rgb=True)


# Create RGB static with fading RGB background hue as gif at 30 fps

In [11]:

Y = 1080
X = 1980

alpha = Image.new('1', (X, Y))
draw = ImageDraw.Draw(alpha)
draw.ellipse((50, 50, 450, 450), fill='blue', outline='blue')
alpha_inv = ImageOps.invert(alpha.convert('RGB'))
alpha_inv = alpha_inv.convert('1')

gif = []
for hue in range(360):
    color = ImageColor.getrgb(f'hsv({hue}, 100%, 100%)')
    background = Image.new('RGBA', (X, Y), color)

    static = gen_frame(gen_noise(Y, X, rgb=True), rgb=True)

    blended = Image.blend(background, static, 0.9)
    # blended.putalpha(alpha_inv)

    # static.putalpha(alpha)

    static_bw = gen_frame(gen_noise(Y, X, rgb=False), rgb=False)
    # blended.paste(static_bw, (0, 0), alpha)

    # blended = blended.convert('L')

    # blended.save(f'./images/img{hue:03d}.png')
    gif.append(blended)
    # blended.show()

save_gif(gif, 360, 30, rgb=True)


In [ ]:
# cat = Image.open('/Users/andrewteta/Desktop/IMG_0706-4.bmp')
# cat = cat.rotate(-90)
# cat.show()


In [ ]:
# import cv2

image_folder = './images'
video_name = 'static_rgb30.mp4'

images = [img for img in os.listdir(image_folder) if img.endswith(".png")]
images.sort()
frame = cv2.imread(os.path.join(image_folder, images[0]))
height, width, layers = frame.shape

fourcc = cv2.VideoWriter_fourcc(*'avc1')
video = cv2.VideoWriter(video_name, fourcc, 30, (width, height))

for image in images:
    video.write(cv2.imread(os.path.join(image_folder, image)))

cv2.destroyAllWindows()
video.release()
